## Battle of Neighborhood
This note is part of the IBM Applied Data Science Capstone project. This course is Part-4 of 4-Course Specialization. The capstone project involves using Python and libraries such as numpy, pandas, matplotlib, seaborn, and folium. The Project is 'Battle of neighborhood'.<br>
Lets say, you want to relocate to another city, due to work reasons. You adore your current neighbourhood, and lounge for same/similar neighbourhood in your new city. This project, involves finding the right town using the location data offerred by Foursquare API. Let's dive in... 

In [270]:
#conda install -c conda-forge geopy
#conda install -c conda-forge geocoder

In [303]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
import os
import folium 
from geopy.geocoders import Nominatim
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### Web Scraping using Beautiful Soup
Scraping a table from wikipdeia page (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) using Beautiful soup. Converting the scraped table into a dataframe using read_html function of pandas. 

In [272]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')
lis = pd.read_html(URL)[0]
df = pd.DataFrame(lis)
df.head()


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Removing unnecessary data
Removing Not assigned Borough from the dataframe

In [273]:
df = df[df['Borough'] != 'Not assigned']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Updating Not Assigned Neighbourhoods
Updating  Not Assigned Neighbourhoods as corresponding Borough. 

In [274]:
# Using np.where to update 'Not assigned' neighbourhood to Borough, and keeping the same value - otherwise
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])

In [275]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [276]:
df.shape

(103, 3)

### Getting Latitudes and Longitudes
Using the Geospacial_Coordinates.csv to get the latitude and longitude of each postal code

In [277]:
path = os.getcwd()
df_code = pd.read_csv(path +'\Geospatial_Coordinates.csv')


### Updating the dataframe with Latitude and Longitude
Getting the latitudes and longitudes for every Postal code, and updating the row of each dataframe with corresponding latitude and longitude.

In [278]:
def get_lat_long(x, df_code):
    for index, i in enumerate(df_code['Postal Code']):
        if i == x:
            return df_code.iloc[index][['Latitude', 'Longitude']]
    return ([np.nan, np.nan])

df[['Latitude', 'Longitude']] = df['Postal Code'].apply(get_lat_long, df_code = df_code)
df.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Neighbourhoods in Toronto
Extracting Neighbourhoods in Toronto

In [279]:
df = df[df['Borough'].str.contains('Toronto')]
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
30,M4E,East Toronto,The Beaches,43.676357,-79.293031


Visualizing the Neighbourhoods of Toronto using Follium

In [280]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [281]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Get top 100 venues from each neighbourhood
Get top 100 venues using Foursquare API for each neighbourhood.<br>
Convert it into a dataframe with following columns: 'Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'. <br>

In [282]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [283]:
def getVenues(neighbourhood, latitude, longitude, radius = 500):
    venues_list = []
    CLIENT_ID = 'CCTF4NNBYAWDVLMNWIZ0BJCQFPH1GPULI3MAZKUKH14SMHIK'
    CLIENT_SECRET = 'JZNFEJ2Y500N12LM5H0QDZ1PSGHNMZ0P1GGLEA3XXHFV3BDD'
    VERSION = '20180605'
    LIMIT = 100
    for name, lat, lng in zip(neighbourhood, latitude, longitude):
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                    CLIENT_ID, 
                    CLIENT_SECRET, 
                    VERSION, 
                    lat, 
                    lng, 
                    radius, 
                    LIMIT)
         # make the GET request
        try:
            results = requests.get(url).json()['response']['groups'][0]['items']
        except:
            continue
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return nearby_venues

In [284]:
toronto_venues = getVenues(neighbourhood=df['Neighbourhood'],
                                   latitude=df['Latitude'],
                                   longitude=df['Longitude']
                                  )
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
...,...,...,...,...,...,...,...
1619,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,Park
1620,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio
1621,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station
1622,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Greenwood Cigar & Variety,43.664538,-79.325379,Smoke Shop


### Group by neighbourhood to view number of venues at each neighbourhood

In [285]:
grp_df = toronto_venues.groupby('Neighborhood')['Venue'].count()
grp_df

Neighborhood
Berczy Park                                                                                                    58
Brockton, Parkdale Village, Exhibition Place                                                                   22
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto                           17
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport     16
Central Bay Street                                                                                             65
Christie                                                                                                       16
Church and Wellesley                                                                                           79
Commerce Court, Victoria Hotel                                                                                100
Davisville                                                                 

### Finding the most popular venues at each neighbourhood
One hot encode the Venue Categories<br>
Find mean of each category across every neighbourhood to find the top 5 venues at each neighbourhood
Draw a dataframe to display the top 5 venues of every neighbourhood. <br>
Note: Some venues would have None value, as its mean = 0

In [286]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [287]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.058824,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.015385,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.015385,0.000000,0.000000,0.015385,0.000000
5,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.025316,0.012658,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012658,...,0.012658,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.012658,0.000000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
8,Davisville,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.026316,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [288]:
toronto_grouped = toronto_grouped.transpose()
new_header = toronto_grouped.iloc[0] #grab the first row for the header
toronto_grouped = toronto_grouped[1:] #take the data less the header row
toronto_grouped.columns = new_header #set the header row as the df header
toronto_grouped.head()

Neighborhood,Berczy Park,"Brockton, Parkdale Village, Exhibition Place","Business reply mail Processing Centre, South Central Letter Processing Plant Toronto","CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",Central Bay Street,Christie,Church and Wellesley,"Commerce Court, Victoria Hotel",Davisville,Davisville North,...,St. James Town,"St. James Town, Cabbagetown",Stn A PO Boxes,Studio District,"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park","The Annex, North Midtown, Yorkville",The Beaches,"The Danforth West, Riverdale","Toronto Dominion Centre, Design Exchange","University of Toronto, Harbord"
Yoga Studio,0,0,0.0588235,0,0.0153846,0,0.0253165,0,0,0,...,0,0,0.010101,0.027027,0,0,0,0.0232558,0,0.0294118
Adult Boutique,0,0,0,0,0,0,0.0126582,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Airport,0,0,0,0.0625,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Airport Food Court,0,0,0,0.0625,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Airport Gate,0,0,0,0.0625,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [289]:
toronto_grouped.transpose()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Berczy Park,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.0172414,0,0,0,0
"Brockton, Parkdale Village, Exhibition Place",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",0.0588235,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",0,0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0,0,...,0,0,0,0,0,0,0,0,0,0
Central Bay Street,0.0153846,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.0153846,0,0,0.0153846,0
Christie,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Church and Wellesley,0.0253165,0.0126582,0,0,0,0,0,0,0.0126582,0,...,0.0126582,0,0,0,0,0,0,0.0126582,0,0
"Commerce Court, Victoria Hotel",0,0,0,0,0,0,0,0,0.04,0,...,0,0,0,0,0,0.02,0,0,0.01,0
Davisville,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0263158,0,0,0,0,0,0,0


In [290]:
lis = []
lis2 = []
num_top_venues = 5
columns = []
for ind, col in toronto_grouped.items():
    lis.append(ind)
    sort = col.sort_values(ascending = False).head(num_top_venues)
    sort = sort[sort.values != 0]
    lis2.append(sort.index.array)
    
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

new_df2 = pd.DataFrame(lis2, columns = columns)
new_df2['Neighborhood'] = lis
cols = list(new_df2.columns)
cols = [cols[-1]] + cols[:-1]
new_df2 = new_df2[cols]
new_df2.shape

(39, 6)

### Clustering

Kmeans clustering with cluster size = 5, and running them on the toronto neighborhood

In [308]:
# set number of clusters
kclusters = 5
#toronto_grouped = toronto_grouped.transpose()
toronto_grouped_clustering = toronto_grouped #.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 1, 4,
       4, 4, 4, 4, 1, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

Adding cluster labels to original dataframe

In [309]:
df['Cluster Label'] = kmeans.labels_
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Label
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4
30,M4E,East Toronto,The Beaches,43.676357,-79.293031,4


Merging original dataframe and the top venues dataframe on neighbourhood

In [312]:
new_df3 = pd.merge(df, new_df2, how = 'inner', left_on = 'Neighbourhood', right_on = 'Neighborhood')
new_df3.head(5)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Label,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4,"Regent Park, Harbourfront",Coffee Shop,Bakery,Park,Pub,Café
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Gym,Discount Store,Park
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Hotel
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,St. James Town,Coffee Shop,Café,Cocktail Bar,American Restaurant,Gym
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,The Beaches,Asian Restaurant,Coffee Shop,Health Food Store,Trail,Pub


### Visualizing on the Map
Visualizing the formed clusters ased on the location on Toronto city map using Folium

In [311]:
#map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(new_df3['Latitude'], new_df3['Longitude'], new_df3['Neighborhood'], new_df3['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters